# Input widgets

To interact with the [HERE search & geocoding API](https://developer.here.com/documentation/geocoding-search-api/api-reference-swagger.html) services `here_search` provides a few widgets based on [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/index.html):
- `SubmittableTextBox` and `TermsButtons` to formulate free-text queries for Autosuggest and Discover
- `OntologyButtons` to formulate ontology shortcuts for Browse

### SubmittableTextBox

`SubmittableTextBox` is a special `Text` widget supporting key strokes events and text submissions.<br/>
Its methods `get_key_stroke_future()` and `get_submitted_value_future()` return an awaitable set to respectively the observed and the validated text.

Both methods are meant to be awaited continuously. In the snippet below, key strokes and text submissions are displayed in an `Output` widget.

In [ ]:
from here_search.widgets.request import SubmittableTextBox
from ipywidgets import Output
from asyncio import ensure_future

box = SubmittableTextBox()
layout_30 = {'height': '30px', 'border': '1px solid black'}
out_keys = Output(layout=layout_30)
out_text = Output(layout=layout_30)

async def print_key_strokes(box, out):
    while True:
        with out:
            keys = await box.get_key_stroke_future()
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': keys}, )

async def print_submitted_text(box, out):
    while True:
        with out:
            text = await box.get_submitted_value_future()
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': text}, )            
            
t1 = ensure_future(print_key_strokes(box, out_keys))
t2 = ensure_future(print_submitted_text(box, out_text))
display(box, out_keys, out_text)

### TermsButtons

`TermsButtons` objects are taking a `SubmittableTextBox` instance to feed it with predefined values when the buttons are selected.<br/>
Per default, the complete form value is overwritten by the button text. 

The constructor `index` parameter allows to specify wich token is overwritten:

In [ ]:
from here_search.widgets.request import TermsButtons

box2 = SubmittableTextBox()
out_keys2 = Output(layout=layout_30)

terms = TermsButtons(box2, 
                     values=["text 1", "text 2"], 
                     index=-1)

t3 = ensure_future(print_key_strokes(box2, out_keys2))
display(box2, terms, out_keys2)

### OntologyButtons

`OntologyButtons` is a widget associating a list of buttons with icons a to a place ontology.<br/>
Its method `get_ontology_future()` returns an awaitable set to the ontology item associated with a selected button.

The example belows defines an ontology and the related buttons. Each buttons selection displays the related ontology in an `Output` widget.

(fontawesome 5.12.0 [download link](https://use.fontawesome.com/releases/v5.12.0/fontawesome-free-5.12.0-web.zip))

In [ ]:
from here_search.entities import OntologyItem, Ontology

items, icons = zip(*[
            (OntologyItem("gas",     ["700-7600-0000", 
                                      "700-7600-0116", 
                                      "700-7600-0444"], None,       None),    "fa-gas-pump"),
            (OntologyItem("eat",     ["100"],           None,       None),    "fa-utensils"),
            (OntologyItem("sleep",   ["500-5000"],      None,       None),    "fa-bed"),
            (OntologyItem("park",    ["400-4300",
                                      "800-8500"],      None,       None),    "fa-parking"),
            (OntologyItem("ATM",     ["700-7010-0108"], None,       None),    "fa-euro-sign"),
            (OntologyItem("pizza",    None,            ["800-057"], None),    "fa-pizza-slice"),
            (OntologyItem("fastfood", None,             None,      ["1566", 
                                                                    "1498"]), "fa-hamburger")])
ontology = Ontology(items)

In [ ]:
from here_search.widgets.request import OntologyButtons
buttons = OntologyButtons(ontology, icons)
out_onto = Output(layout=layout_30)

async def print_ontology(buttons, out):
    while True:
        with out:
            ontology = await buttons.get_ontology_future()
            out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': str(ontology)}, )
            
t4 = ensure_future(print_ontology(buttons, out_onto))
display(buttons, out_onto)

## OneBoxSimple

Similarly to the previous snippets, the `OneBoxSimple` objects continuously wait for three event types:
- key strokes through `wait_for_new_key_stroke()`
- text submissions through `wait_for_submitted_value()`
- ontology selections through `wait_for_selected_ontology()`

`OneBoxSimple` then sends keystrokes, text submissions and ontology selections to Autosuggest, Discover and Browse respectively.<br/>
The related responses are delivered to the two methods:
- Autosuggest results to `handle_suggestion_list()` 
- Discover and Browse results to `handle_result_list()`

In the snippet below, `App` uses 
- `SubmittableTextBox`, `TermsButtons` and `OntologyButtons` widgets to feed the three event types of `OneBoxSimple`
- a `Output` widget to handle the API responses:

In [ ]:
from here_search.base import OneBoxSimple
from asyncio import Future
from json import dumps

text_box = SubmittableTextBox()
terms_buttons = TermsButtons(text_box, index=-1)
ontology_buttons = OntologyButtons(ontology, icons)
out_simple = Output(layout={'height': '160px', 'border': '1px solid black', 'overflow': 'auto', 'white-space': 'nowrap'})


class App(OneBoxSimple):
    default_language = "de"
    
    def __init__(self, ontology_buttons, text_box, terms_buttons, out):
        super().__init__()
        self.text_box = text_box
        self.terms_buttons = terms_buttons
        self.ontology_buttons = ontology_buttons
        self.out = out
    
    def wait_for_new_key_stroke(self):
        return self.text_box.get_key_stroke_future()
    
    def wait_for_submitted_value(self):
        return self.text_box.get_submitted_value_future()
    
    def wait_for_selected_ontology(self):
        return self.ontology_buttons.get_ontology_future()
    
    def handle_suggestion_list(self, resp):
        with self.out:
            self.out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': dumps(resp.data, indent=2)}, )
        text = {term['term']: None for term in resp.data.get('queryTerms', [])}
        self.terms_buttons.set(list(text.keys()))
            
    def handle_result_list(self, resp):
        with self.out:
            self.out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': dumps(resp.data, indent=2)}, )
        self.terms_buttons.set([])
        
    def handle_empty_text_submission(self):
        with self.out:
            self.out.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': ""}, )
        self.terms_buttons.set([])

app = App(ontology_buttons, text_box, terms_buttons, out_simple).run()
display(ontology_buttons, text_box, terms_buttons, out_simple)

## PositionMap

`OneBoxSimple` always uses the same `search_center` public property, initialized to the same value `52.51604, 13.37691`. To change this location context, a new widget is needed.

The `PositionMap` class uses [`here-map-widget`](https://pypi.org/project/here-map-widget-for-jupyter/) dragging and zooming capabilities to capture the center of a HERE map and sends it to an observer. This class expects a `position_handler` constructor parameter respecting the traitlets event handler [signature](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html#Registering-callbacks-to-trait-changes-in-the-kernel). In the snippet below, the map position is only displayed in an `Output` widget, each time the user interacts with the map:

In [ ]:
from here_search.widgets.request import PositionMap
from here_search.api import API

out_map = Output(layout = {'width': '100%', 'height': '30px', 'overflow': 'auto', 'white-space': 'nowrap'})

def observe(change):
    if change.type == "change" and change.new:
        with out_map:
            out_map.outputs = ({'name': 'stdout', 'output_type': 'stream', 'text': dumps({"name": change.name, "new": change.new})}, )
    
pmap = PositionMap(api_key=API().api_key, 
                   center=(52.51604, 13.37691),
                   layout = {'height': '200px'},
                   position_handler=observe)
display(pmap, out_map)

we adapt the observer to not display the map center but set the `search_center` property of an `App` instance to it:

In [ ]:
for widget in out_simple, terms_buttons, text_box, ontology_buttons, pmap:
    widget.close()
    del widget

text_box = SubmittableTextBox()
terms_buttons = TermsButtons(text_box, index=-1)
ontology_buttons = OntologyButtons(ontology, icons)
out_simple = Output(layout={'height': '160px', 'border': '1px solid black', 'overflow': 'auto', 'white-space': 'nowrap'})

app = App(ontology_buttons, text_box, terms_buttons, out_simple)

def observe(change):
    if change.type == "change" and change.name == "center":
        app.search_center = change.new[:2]

pmap = PositionMap(api_key=API().api_key, 
           center=(52.51604, 13.37691),
           layout = {'height': '200px'},
           position_handler=observe)

app.run()
display(ontology_buttons, text_box, terms_buttons, pmap, out_simple)

# OneBoxMap


`OneBoxMap` is a demo application showing how [HERE Geocoding and Search](https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics/endpoint-autosuggest-brief.html) `/autosuggest`, `/discover` and `/lookup` endpoints are meant to be used:
- The app  
    - proposes a single text form to formulate queries
    - displays three buttons with predicted last token text completions
    - displays all API calls in a separate window
    - displays search results on a map and a list
<br/>
- `/autosuggest` endpoint is called for each key-stroke.
    - Follow-up query suggestions are displayed in the list only
    - Location suggestions are displayed in the list and on the map
    - Query term suggestions are displayed in three buttons
    - Selected locations lead to a call to `/lookup` using the location record `id`.
    - Selected query suggestions lead to a http GET using the query `href` value
<br/>
- `/discover` endpoint is called when a query submission is validated.
    - Validation happens when the [return] key or the <img src="https://upload.wikimedia.org/wikipedia/commons/2/2b/Font_Awesome_5_solid_search.svg" style="width:12px"/> button are hit
    - Location results are displayed in the list and on the map
    - Selected results lead to a call to `/lookup` using the location record `id`.
<br/>
- `/autosuggest` and `/discover` requests are sent using the map center as search center
- Search requests are cached during the lifetime of the application


<!-- https://commons.wikimedia.org/wiki/File:Font_Awesome_5_solid_search.svg -->

In [ ]:
from here_search.widgets.app import OneBoxMap
OneBoxMap().run()